In [62]:
import pandas as pd
from datetime import datetime
import matplotlib.pylab as plt
from pmdarima.arima import auto_arima
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,6
import seaborn as srn

In [2]:
dados = pd.read_csv("Dados_vendas.csv")
dados

,ID,ProductID,ClientID,Discount,UnitPrice,Quantity,StoreID,Date
0,80260d682079b6090c8285b398c50d97,002552c0663708129c0019cc97552d7d3,14001,0.08,249.2,1,4,2018-12-10
1,3275736da1234f55d52bdf09d86b93bc,001b237c0e9bb435f2e54071129237e93,14001,0.10,162.4,1,4,2019-04-25
2,f3513eec8f1434b5b7e04ddf3598df40,00066f42aeeb9f3007548bb9d3f33c381,14001,0.10,194.6,1,4,2019-07-17
3,7180475b60fe6bbbde424d39704a1177,001b72dfd63e9833e8c02742adf472e33,14002,0.10,201.6,2,1,2018-04-10
4,76600853a200fe87e141adae6b35674c,00210e41887c2a8ef9f791ebc780cc363,14002,0.10,406.0,1,1,2019-01-21
...,...,...,...,...,...,...,...,...
32748,b155e2a8589d40bc9c0381ee48e3204f,002ec297b1b00fb9dde7ee6ac24b67712,28694,0.08,218.4,1,4,2018-12-02
32749,abb71d0110c4be1a1a0c52f929cab4a3,002d4ea7c04739c130bb74d7e7cd16943,28694,0.08,210.0,1,4,2018-12-05
32750,db9a034f34174edd577b9f64cc982952,001795ec6f1b187d37335e1c4704762e3,28694,0.08,148.4,1,4,2019-08-01
32751,ee00fa5093eb96d8573403b382fb60a8,002ec297b1b00fb9dde7ee6ac24b67713,28694,0.10,218.4,2,4,2019-11-25


In [3]:
# Removi as colunar ID e ClientID pois não eram necessárias nesse momento
dados.drop("ID", axis=1, inplace=True)
dados.drop("ClientID", axis=1, inplace=True)

In [4]:
# Aqui criei uma nova coluna chamada Valor Total passando como valor uma
# expressão matemática que multiplica a quantidade pelo preço unitário
# e depois multiplica pelo desconto menos 1
dados["Valor Total"] = (dados["UnitPrice"] * dados["Quantity"]) * (1 - dados["Discount"])
dados["Valor Total"] = dados["Valor Total"].round(2)

In [5]:
dados

,ProductID,Discount,UnitPrice,Quantity,StoreID,Date,Valor Total
0,002552c0663708129c0019cc97552d7d3,0.08,249.2,1,4,2018-12-10,229.26
1,001b237c0e9bb435f2e54071129237e93,0.10,162.4,1,4,2019-04-25,146.16
2,00066f42aeeb9f3007548bb9d3f33c381,0.10,194.6,1,4,2019-07-17,175.14
3,001b72dfd63e9833e8c02742adf472e33,0.10,201.6,2,1,2018-04-10,362.88
4,00210e41887c2a8ef9f791ebc780cc363,0.10,406.0,1,1,2019-01-21,365.40
...,...,...,...,...,...,...,...
32748,002ec297b1b00fb9dde7ee6ac24b67712,0.08,218.4,1,4,2018-12-02,200.93
32749,002d4ea7c04739c130bb74d7e7cd16943,0.08,210.0,1,4,2018-12-05,193.20
32750,001795ec6f1b187d37335e1c4704762e3,0.08,148.4,1,4,2019-08-01,136.53
32751,002ec297b1b00fb9dde7ee6ac24b67713,0.10,218.4,2,4,2019-11-25,393.12


In [6]:
# Aqui carreguei o arquivo onde estavam os nomes e tamanhos dos produtos
id_produtos = pd.read_csv("Dados_produtos.csv")
id_produtos

,ID,Name,Size
0,00066f42aeeb9f3007548bb9d3f33c381,Capitão América,P
1,00066f42aeeb9f3007548bb9d3f33c382,Capitão América,M
2,00066f42aeeb9f3007548bb9d3f33c383,Capitão América,G
3,00088930e925c41fd95ebfe695fd26551,Thor,P
4,00088930e925c41fd95ebfe695fd26552,Thor,M
...,...,...,...
68,0030e635639c898b323826589761cf233,Rick Morty,G
69,003128f981470c3e5a2e7445e4a771cd1,Homer Simpsons,P
70,003128f981470c3e5a2e7445e4a771cd2,Homer Simpsons,M
71,003128f981470c3e5a2e7445e4a771cd3,Homer Simpsons,G


In [7]:
# Aqui fiz um merge para juntar as tabelas, fazendo um inner na coluna Product e ID
# para poder vizualizar o nome dos produtos e não mais o ID
dados = pd.merge(dados,id_produtos,left_on="ProductID",right_on="ID",how="inner")

In [8]:
dados

,ProductID,Discount,UnitPrice,Quantity,StoreID,Date,Valor Total,ID,Name,Size
0,002552c0663708129c0019cc97552d7d3,0.08,249.2,1,4,2018-12-10,229.26,002552c0663708129c0019cc97552d7d3,Tempestade,G
1,002552c0663708129c0019cc97552d7d3,0.10,249.2,1,1,2019-03-28,224.28,002552c0663708129c0019cc97552d7d3,Tempestade,G
2,002552c0663708129c0019cc97552d7d3,0.08,249.2,1,2,2019-08-01,229.26,002552c0663708129c0019cc97552d7d3,Tempestade,G
3,002552c0663708129c0019cc97552d7d3,0.08,249.2,1,7,2019-06-04,229.26,002552c0663708129c0019cc97552d7d3,Tempestade,G
4,002552c0663708129c0019cc97552d7d3,0.08,249.2,1,2,2019-08-01,229.26,002552c0663708129c0019cc97552d7d3,Tempestade,G
...,...,...,...,...,...,...,...,...,...,...
32748,001795ec6f1b187d37335e1c4704762e1,0.10,148.4,1,4,2018-10-30,133.56,001795ec6f1b187d37335e1c4704762e1,Mulher-Maravilha,P
32749,001795ec6f1b187d37335e1c4704762e1,0.08,148.4,1,7,2019-08-10,136.53,001795ec6f1b187d37335e1c4704762e1,Mulher-Maravilha,P
32750,001795ec6f1b187d37335e1c4704762e1,0.08,148.4,1,4,2018-08-09,136.53,001795ec6f1b187d37335e1c4704762e1,Mulher-Maravilha,P
32751,001795ec6f1b187d37335e1c4704762e1,0.08,148.4,1,7,2019-08-17,136.53,001795ec6f1b187d37335e1c4704762e1,Mulher-Maravilha,P


In [9]:
# Aqui exclui as colunas ProductID, ID, Discount e Unitprice
# pois não eram mais necessárias
dados.drop("ProductID", axis=1, inplace=True)
dados.drop("ID", axis=1, inplace=True)

In [10]:
dados.drop("Discount", axis=1, inplace=True)
dados.drop("UnitPrice", axis=1, inplace=True)

In [11]:
dados

,Quantity,StoreID,Date,Valor Total,Name,Size
0,1,4,2018-12-10,229.26,Tempestade,G
1,1,1,2019-03-28,224.28,Tempestade,G
2,1,2,2019-08-01,229.26,Tempestade,G
3,1,7,2019-06-04,229.26,Tempestade,G
4,1,2,2019-08-01,229.26,Tempestade,G
...,...,...,...,...,...,...
32748,1,4,2018-10-30,133.56,Mulher-Maravilha,P
32749,1,7,2019-08-10,136.53,Mulher-Maravilha,P
32750,1,4,2018-08-09,136.53,Mulher-Maravilha,P
32751,1,7,2019-08-17,136.53,Mulher-Maravilha,P


In [12]:
# Aqui estou criando uma coluna chamada Produtos unindo as colunas Name e Size
# Com isso posso excluir as colunas Name e Size
dados["Produtos"] = dados["Name"] + " - " + dados["Size"]
dados.drop("Name", axis=1, inplace=True)
dados.drop("Size", axis=1, inplace=True)

In [13]:
dados

,Quantity,StoreID,Date,Valor Total,Produtos
0,1,4,2018-12-10,229.26,Tempestade - G
1,1,1,2019-03-28,224.28,Tempestade - G
2,1,2,2019-08-01,229.26,Tempestade - G
3,1,7,2019-06-04,229.26,Tempestade - G
4,1,2,2019-08-01,229.26,Tempestade - G
...,...,...,...,...,...
32748,1,4,2018-10-30,133.56,Mulher-Maravilha - P
32749,1,7,2019-08-10,136.53,Mulher-Maravilha - P
32750,1,4,2018-08-09,136.53,Mulher-Maravilha - P
32751,1,7,2019-08-17,136.53,Mulher-Maravilha - P


In [14]:
# Fiz um rename nas colunas só para ficar mais apresentável
dados.rename(columns={"Quantity":"Quantidade", "StoreID":"Loja","Date":"Data"}, inplace=True)

In [15]:
dados

,Quantidade,Loja,Data,Valor Total,Produtos
0,1,4,2018-12-10,229.26,Tempestade - G
1,1,1,2019-03-28,224.28,Tempestade - G
2,1,2,2019-08-01,229.26,Tempestade - G
3,1,7,2019-06-04,229.26,Tempestade - G
4,1,2,2019-08-01,229.26,Tempestade - G
...,...,...,...,...,...
32748,1,4,2018-10-30,133.56,Mulher-Maravilha - P
32749,1,7,2019-08-10,136.53,Mulher-Maravilha - P
32750,1,4,2018-08-09,136.53,Mulher-Maravilha - P
32751,1,7,2019-08-17,136.53,Mulher-Maravilha - P


In [16]:
# Aqui realizo mais um carregamento para vizualizar o nome das lojas
lojas_nome = pd.read_csv("Dados_lojas.csv")
dados = pd.merge(dados, lojas_nome, left_on="Loja", right_on="ID", how="inner")

In [17]:
dados

,Quantidade,Loja,Data,Valor Total,Produtos,ID,Name,State
0,1,4,2018-12-10,229.26,Tempestade - G,4,Curitiba,PR
1,1,4,2018-12-03,229.26,Tempestade - G,4,Curitiba,PR
2,1,4,2018-04-27,224.28,Tempestade - G,4,Curitiba,PR
3,1,4,2019-02-21,224.28,Tempestade - G,4,Curitiba,PR
4,1,4,2018-10-01,224.28,Tempestade - G,4,Curitiba,PR
...,...,...,...,...,...,...,...,...
32748,1,22,2019-02-14,133.56,Mulher-Maravilha - P,22,Online,na
32749,1,22,2019-12-22,136.53,Mulher-Maravilha - P,22,Online,na
32750,1,22,2020-01-01,133.56,Mulher-Maravilha - P,22,Online,na
32751,2,22,2018-08-18,273.06,Mulher-Maravilha - P,22,Online,na


In [18]:
# Aqui eu excluo as colunas Loja, IDm e State, pois não são mais necessárias.
dados.drop("Loja", axis=1, inplace=True)
dados.drop("ID", axis=1, inplace=True)

In [19]:
dados

,Quantidade,Data,Valor Total,Produtos,Name,State
0,1,2018-12-10,229.26,Tempestade - G,Curitiba,PR
1,1,2018-12-03,229.26,Tempestade - G,Curitiba,PR
2,1,2018-04-27,224.28,Tempestade - G,Curitiba,PR
3,1,2019-02-21,224.28,Tempestade - G,Curitiba,PR
4,1,2018-10-01,224.28,Tempestade - G,Curitiba,PR
...,...,...,...,...,...,...
32748,1,2019-02-14,133.56,Mulher-Maravilha - P,Online,na
32749,1,2019-12-22,136.53,Mulher-Maravilha - P,Online,na
32750,1,2020-01-01,133.56,Mulher-Maravilha - P,Online,na
32751,2,2018-08-18,273.06,Mulher-Maravilha - P,Online,na


In [20]:
dados.rename(columns={"Name":"Loja"}, inplace=True)

In [21]:
# Aqui vamos unir as colunas Loja e State em apenas uma.
# Podemos excluir a coluna State
dados["Loja"] = dados["Loja"] + " - " + dados["State"]
dados.drop("State", axis=1, inplace=True)

In [22]:
dados

,Quantidade,Data,Valor Total,Produtos,Loja
0,1,2018-12-10,229.26,Tempestade - G,Curitiba - PR
1,1,2018-12-03,229.26,Tempestade - G,Curitiba - PR
2,1,2018-04-27,224.28,Tempestade - G,Curitiba - PR
3,1,2019-02-21,224.28,Tempestade - G,Curitiba - PR
4,1,2018-10-01,224.28,Tempestade - G,Curitiba - PR
...,...,...,...,...,...
32748,1,2019-02-14,133.56,Mulher-Maravilha - P,Online - na
32749,1,2019-12-22,136.53,Mulher-Maravilha - P,Online - na
32750,1,2020-01-01,133.56,Mulher-Maravilha - P,Online - na
32751,2,2018-08-18,273.06,Mulher-Maravilha - P,Online - na


In [23]:
# Aqui eu mudo a ordem de apresentação das colunas
dados = dados[["Data","Produtos","Loja","Quantidade","Valor Total"]]
dados

,Data,Produtos,Loja,Quantidade,Valor Total
0,2018-12-10,Tempestade - G,Curitiba - PR,1,229.26
1,2018-12-03,Tempestade - G,Curitiba - PR,1,229.26
2,2018-04-27,Tempestade - G,Curitiba - PR,1,224.28
3,2019-02-21,Tempestade - G,Curitiba - PR,1,224.28
4,2018-10-01,Tempestade - G,Curitiba - PR,1,224.28
...,...,...,...,...,...
32748,2019-02-14,Mulher-Maravilha - P,Online - na,1,133.56
32749,2019-12-22,Mulher-Maravilha - P,Online - na,1,136.53
32750,2020-01-01,Mulher-Maravilha - P,Online - na,1,133.56
32751,2018-08-18,Mulher-Maravilha - P,Online - na,2,273.06


In [24]:
dados

,Data,Produtos,Loja,Quantidade,Valor Total
0,2018-12-10,Tempestade - G,Curitiba - PR,1,229.26
1,2018-12-03,Tempestade - G,Curitiba - PR,1,229.26
2,2018-04-27,Tempestade - G,Curitiba - PR,1,224.28
3,2019-02-21,Tempestade - G,Curitiba - PR,1,224.28
4,2018-10-01,Tempestade - G,Curitiba - PR,1,224.28
...,...,...,...,...,...
32748,2019-02-14,Mulher-Maravilha - P,Online - na,1,133.56
32749,2019-12-22,Mulher-Maravilha - P,Online - na,1,136.53
32750,2020-01-01,Mulher-Maravilha - P,Online - na,1,133.56
32751,2018-08-18,Mulher-Maravilha - P,Online - na,2,273.06


In [25]:
# Transformei a coluna Data que era um obejeto em Datetime
dados["Data"] = pd.to_datetime(dados["Data"], format="%Y-%m-%d")

In [26]:
# Ordenei pela coluna Data
dados = dados.sort_values(by="Data")

In [27]:
dados

,Data,Produtos,Loja,Quantidade,Valor Total
16260,2018-03-01,Flash - G,Rio de Janeiro - RJ,1,183.96
30697,2018-03-01,Pantera Negra - G,Belo Horizonte - MG,1,181.44
27285,2018-03-01,Flash - P,Porto Alegre - RS,1,183.96
22478,2018-03-01,Homer Simpsons - M,São Paulo - SP,2,289.80
16087,2018-03-01,Flash - G,Rio de Janeiro - RJ,1,183.96
...,...,...,...,...,...
31708,2020-01-31,Deadpool - M,Belo Horizonte - MG,1,229.32
25261,2020-01-31,Darth Vader - G,Porto Alegre - RS,1,133.56
11761,2020-01-31,Dragon Ball Z - P,Florianópolis - SC,1,120.96
19868,2020-01-31,Capitã Marvel - M,Rio de Janeiro - RJ,1,204.12


In [28]:
# Aqui eu coloquei a data como indice do meu DataFrame
dados = dados.set_index("Data")

In [29]:
dados

,Produtos,Loja,Quantidade,Valor Total
Data,,,,
2018-03-01,Flash - G,Rio de Janeiro - RJ,1,183.96
2018-03-01,Pantera Negra - G,Belo Horizonte - MG,1,181.44
2018-03-01,Flash - P,Porto Alegre - RS,1,183.96
2018-03-01,Homer Simpsons - M,São Paulo - SP,2,289.80
2018-03-01,Flash - G,Rio de Janeiro - RJ,1,183.96
...,...,...,...,...
2020-01-31,Deadpool - M,Belo Horizonte - MG,1,229.32
2020-01-31,Darth Vader - G,Porto Alegre - RS,1,133.56
2020-01-31,Dragon Ball Z - P,Florianópolis - SC,1,120.96


In [88]:
# Salvei em um arquivo xlsx para apresentar no power Bi
dados.to_excel("Dados_vendas.xlsx")

In [101]:
# Faturamento total por ano
faturamento = dados.iloc[:,[3]]
faturamento = faturamento.reset_index()
faturamento = faturamento.groupby(pd.Grouper(key="Data", freq="Y")).sum()
faturamento

,Valor Total
Data,
2018-12-31,3289666.91
2019-12-31,3531187.01
2020-12-31,153009.36


In [79]:
# Produto que mais vendeu, considerando todos os anos
produtos_geral = dados.iloc[:,[0,2]]
produtos_geral = produtos_geral.groupby(["Produtos"]).sum()
produtos_geral = produtos_geral.sort_values(by="Quantidade", ascending=False)
produtos_geral

,Quantidade
Produtos,
Bazinga - G,1020
Iron Man - G,1018
Deadpool - G,998
Capitã Marvel - G,996
Flash - G,996
...,...
Coringa - M,216
Naruto - M,214
Tempestade - M,214


In [58]:
# Produtos com maior faturamento
produtos_fat = dados.iloc[:,[0,3]]
produtos_fat = produtos_fat.groupby(["Produtos"]).sum()
produtos_fat.sort_values(by="Valor Total", ascending=False)

,Valor Total
Produtos,
BIXDream - G,592319.84
Naruto - G,335648.32
Ciclope - G,230893.10
Deadpool - G,230753.22
Tempestade - G,214475.36
...,...
Darth Vader - M,29068.74
Coringa - P,24677.70
Thor - P,24023.15


In [59]:
# Loja que mais vendeu
loja_geral = dados.iloc[:,[1,2]]
loja_geral = loja_geral.groupby(["Loja"]).sum()
loja_geral.sort_values(by="Quantidade", ascending=False)

,Quantidade
Loja,
Florianópolis - SC,8734
Rio de Janeiro - RJ,7224
Porto Alegre - RS,6516
Curitiba - PR,5481
São Paulo - SP,3794
Belo Horizonte - MG,1956
Online - na,969


In [31]:
# Separei os dados por ano
ano_2018 = dados.loc['2018-01-01':'2018-12-31']

In [33]:
ano_2019 = dados.loc['2019-01-01':'2019-12-31']

In [35]:
ano_2020 = dados.loc['2020-01-01':'2020-12-31']

In [60]:
# Aqui vou trazer os produtos mais vendidos de cada ano
produtos_2018 = ano_2018.iloc[:,[0,2]]
produtos_2018 = produtos_2018.groupby(["Produtos"]).sum()
produtos_2018.sort_values(by="Quantidade", ascending=False)

,Quantidade
Produtos,
Thanos - G,601
Deadpool - G,597
Capitã Marvel - G,592
Super Mario - G,587
Flash - G,575
...,...
BIXDream - G,57
Naruto - M,19
Naruto - P,16


In [61]:
produtos_2019 = ano_2019.iloc[:,[0,2]]
produtos_2019 = produtos_2019.groupby(["Produtos"]).sum()
produtos_2019.sort_values(by="Quantidade", ascending=False)

,Quantidade
Produtos,
BIXDream - G,869
Naruto - G,838
Iron Man - G,441
Bazinga - G,437
Dragon Ball Z - G,416
...,...
Pantera Negra - P,81
Capitão América - M,81
Tempestade - P,78
